In [7]:
import pandas as pd

data = {
    'Mixture': ['Caf', 'Ace', 'Asp'],
    'Boiling_temp_(°C)': [178, 332.7, 246],
    'logP': [-0.07, -1.33, 0.07],
    'pKa': [[14], [3.02], [1.08, 9.13]],
    'Molecular_mass': [194, 204, 133],
}

df = pd.DataFrame(data)

def det_chromato(df):
    boiling_temps = [temp for temp in df['Boiling_temp_(°C)'] if temp is not None]
    if boiling_temps and min(boiling_temps) >= 300:
        Chromato_type = 'GC'
        eluent_nature = 'gas'
        proposed_pH = None
    else:
        molar_masses = [mass for mass in df['Molecular_mass'] if mass is not None]
        max_molar_mass = max(molar_masses) if molar_masses else None

        min_pKa = float('inf')  # Initialize min_pKa with a large value
        max_pKa = float('-inf')  # Initialize max_pKa with a small value
        for pKa_entry in df['pKa']:
            if isinstance(pKa_entry, list):
                for pKa_value in pKa_entry:
                    if pKa_value is not None:
                        min_pKa = min(pKa_value, min_pKa)
                        max_pKa = max(pKa_value, max_pKa)
            else:
                if pKa_entry is not None:
                    min_pKa = min(pKa_entry, min_pKa)
                    max_pKa = max(pKa_entry, max_pKa)
        
        if min_pKa == float('inf'):
            min_pKa = None
        if max_pKa == float('-inf'):
            max_pKa = None

        logPs = [logP for logP in df['logP'] if logP is not None]
        max_logP = max(logPs) if logPs else None
        min_logP = min(logPs) if logPs else None

        if max_molar_mass is not None and max_molar_mass <= 2000:
            if max_logP is not None and max_logP < 0:
                proposed_pH = max_pKa + 2 if max_pKa is not None else None
                if proposed_pH is not None and 3 <= proposed_pH <= 11:
                    Chromato_type = 'IC'
                    eluent_nature = 'aqueous'
                else:
                    Chromato_type = 'HPLC'
                    eluent_nature = 'organic or hydro-organic'
                    proposed_pH = min_pKa + 2 if min_pKa is not None else None
            else:
                Chromato_type = 'HPLC'
                if min_logP is not None and -2 <= min_logP <= 0:
                    eluent_nature = 'organic or hydro-organic'
                    if min_logP >= 0:
                        Chromato_type += ' on normal stationary phase'
                    else:
                        Chromato_type += ' on reverse stationary phase using C18 column'
                else:
                    eluent_nature = 'organic or hydro-organic'
                    Chromato_type += ' on normal stationary phase'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
        else:
            if max_logP is not None and max_logP < 0:
                Chromato_type = 'HPLC on reverse stationary phase'
                eluent_nature = 'organic or hydro-organic'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
            else:
                if max_logP is not None and max_logP > 0:
                    Chromato_type = 'SEC on gel permeation with a hydrophobe organic polymer stationary phase'
                    eluent_nature = 'organic solvent'
                else:
                    Chromato_type = 'SEC on gel filtration with a polyhydroxylated hydrophile polymer stationary phase'
                    eluent_nature = 'aqueous'
                proposed_pH = min_pKa + 2 if min_pKa is not None else None
    
    return Chromato_type, eluent_nature, proposed_pH

Mixture_chromato_type, eluent_nature, proposed_pH = det_chromato(df)

print(f"The advisable chromatography type is: {Mixture_chromato_type}")
print(f"Eluent nature: {eluent_nature}")
if proposed_pH is not None:
    print(f"Proposed pH for the eluent: {proposed_pH}")


The advisable chromatography type is: HPLC on reverse stationary phase using C18 column
Eluent nature: organic or hydro-organic
Proposed pH for the eluent: 3.08
